In [ ]:
from google.colab import drive
drive.mount('/content/gdrive') #, force_remount=True

Mounted at /content/gdrive


In [ ]:
import os
os.chdir('gdrive/My Drive/Colab Notebooks/miramedix')

In [ ]:
import urllib.request
from bs4 import BeautifulSoup
from datetime import datetime
import time


In [ ]:
import json

with open('descriptions.json', 'r') as f:
  texts = json.load(f)

# rerankings data

Загрузка переранжированных статей

In [ ]:
import json

all_new_recs = {}

for i in range(len(texts)):
  idx = str(i+1)
  file_name = f'reranking_2/rerank_{idx}.json'
  with open(file_name, 'r') as f:
    all_new_recs[idx] = json.load(f)

In [ ]:
with open('new_recs_raw.json', 'w') as f:
  json.dump(all_new_recs, f)

In [ ]:
with open('new_recs_raw.json', 'r') as f:
  all_new_recs = json.load(f)

In [ ]:
rec_nums = {}
for i in all_new_recs:
  recs_clean = []
  for rec in all_new_recs[i]:
    recs_clean.append(rec[0][3:])
  rec_nums[i] = recs_clean

In [ ]:
rec_nums['1']

['1', '16', '17', '3', '8', '18', '2', '7', '20', '9']

In [ ]:
import json

#with open('texts_recs_with_urls.json', 'r') as f:
#with open('texts_recs_umls_m3_2_clean_reranked.json', 'r') as f:
with open('texts_recs_umls_m3_2_clean.json', 'r') as f:
  data = json.load(f)

In [ ]:
rer_recs = {}

for key in rec_nums.keys():

  text_recs = []  
  for num in rec_nums[key]:
    paper_id = int(num) - 1
    text_recs.append(data[key][paper_id])  
  
  rer_recs[key] = text_recs

In [ ]:
rer_recs['1']

['<h1>Friedreich Ataxia</h1> \nhttps://medlineplus.gov/friedreichataxia.html\n<p>Friedreich ataxia is an inherited disease that damages your nervous system. The damage affects your spinal cord and the nerves that control muscle movement in your arms and legs.  Symptoms usually begin between the ages of 5 and 15. The main symptom is ataxia, which means trouble coordinating movements. Specific symptoms include:</p><ul><li><span class="qt0">Difficulty</span> <span class="qt1">walking</span></li><li>\tMuscle weakness</li><li>\tSpeech problems</li><li>\tInvoluntary eye movements</li><li>\tScoliosis (curving of the spine to one side)</li><li>\tHeart palpitations, from the heart disease which can happen along with Friedreich ataxia</li></ul><p>People with Friedreich ataxia usually need a wheelchair 15 to 20 years after symptoms first appear. In severe cases, people become incapacitated. There is no cure. You can treat symptoms with medicines, braces, surgery, and physical therapy.</p><p>NIH: 

In [ ]:
with open('texts_recs_umls_m3_2_clean_reranked_2ver.json', 'w') as f:
  json.dump(rer_recs, f)

# for label studio

In [ ]:
import json

#with open('texts_recs_with_urls.json', 'r') as f:
with open('texts_recs_umls_m3_2_clean_reranked_2ver.json', 'r') as f:
  data = json.load(f)

In [ ]:
import re

titles = {}
for key in data.keys():
  clean_titles = []
  clean_tag = re.compile('<.*?>') 
  for info in data[key]:
    title_i = info.split('\n')[0]
    url = info.split('\n')[1]
    clean_title = re.sub(clean_tag, '', title_i)
    clean_titles.append(clean_title + '\n' + url)
  titles[key] = clean_titles

In [ ]:
for i in titles.keys():
  print(i)
  print(titles[i])
  print(len(titles[i]))
  break

1
['Friedreich Ataxia \nhttps://medlineplus.gov/friedreichataxia.html', 'Sciatica \nhttps://medlineplus.gov/sciatica.html', 'Spinal Muscular Atrophy \nhttps://medlineplus.gov/spinalmuscularatrophy.html', 'Transient Ischemic Attack \nhttps://medlineplus.gov/transientischemicattack.html', 'Pulmonary Hypertension \nhttps://medlineplus.gov/pulmonaryhypertension.html', 'Chronic Pain \nhttps://medlineplus.gov/chronicpain.html', 'Knee Injuries and Disorders \nhttps://medlineplus.gov/kneeinjuriesanddisorders.html', 'High Blood Pressure \nhttps://medlineplus.gov/highbloodpressure.html', 'Pain \nhttps://medlineplus.gov/pain.html', 'How to Prevent High Blood Pressure \nhttps://medlineplus.gov/howtopreventhighbloodpressure.html']
10


In [ ]:
#for reranked
"""new_titles = {}

for i in titles.keys():
  new_recs = []
  for num in rec_nums[i]:
    rec_ids = int(num)-1
    new_recs.append(titles[i][rec_ids])
  new_titles[i] = new_recs"""

In [ ]:
all_json_data = []

for key in data.keys():
  text_id = key
  query = texts[key]
  rec_titles = titles[key]

  key_temp = {}
  key_temp['id'] = text_id
  data_temp = {} #data key
  annot_temp = {} #annotations key
  all_key_results = []
  heads = query + '\n' + "Select document related to the query:" + '\n\n'
  text_num = 1
  for i in rec_titles[:10]:
    res_dict = {}
    key_name = "text" + str(text_num)
    heads = heads + str(text_num) + '. ' + i + '\n\n'
    text_num += 1

  data_temp['my_text'] = heads
  key_temp['data'] = data_temp

  all_json_data.append(key_temp)

In [ ]:
with open('data_formarted_umls_m32_re_2ver.json', 'w') as f: #umls+m32 reranked
  json.dump(all_json_data, f)

In [ ]:
with open('data_formarted_umls_m32_re.json', 'w') as f: #umls+m32 reranked
  json.dump(all_json_data, f)

In [ ]:
with open('data_formarted_with_urls_8.json', 'w') as f: #original
  json.dump(all_json_data, f)

In [ ]:
with open('data_formarted_with_urls_2ver.json', 'w') as f: #reranked
  json.dump(all_json_data, f)

In [ ]:
for key in all_json_data:
  print(key)
  print(key['data'].keys())
  break

{'id': '1', 'data': {'my_text': '\nPatient is a 45-year-old man with a history of anaplastic astrocytoma of the spine complicated by severe lower extremity weakness and urinary retention s/p Foley catheter, high-dose steroids, hypertension, and chronic pain. The tumor is located in the T-L spine, unresectable anaplastic astrocytoma s/p radiation. Complicated by progressive lower extremity weakness and urinary retention. Patient initially presented with RLE weakness where his right knee gave out with difficulty walking and right anterior thigh numbness. MRI showed a spinal cord conus mass which was biopsied and found to be anaplastic astrocytoma. Therapy included field radiation t10-l1 followed by 11 cycles of temozolomide 7 days on and 7 days off. This was followed by CPT-11 Weekly x4 with Avastin Q2 weeks/ 2 weeks rest and repeat cycle. \n\nSelect document related to the query:\n\n1. High Blood Pressure \nhttps://medlineplus.gov/highbloodpressure.html\n\n2. Pulmonary Hypertension \nht

# for pyserini

In [ ]:
pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json

with open('texts_recs_umls_m3_2_clean.json', 'r') as f:
  data = json.load(f)

In [ ]:
data['1']

['<h1>Friedreich Ataxia</h1> \nhttps://medlineplus.gov/friedreichataxia.html\n<p>Friedreich ataxia is an inherited disease that damages your nervous system. The damage affects your spinal cord and the nerves that control muscle movement in your arms and legs.  Symptoms usually begin between the ages of 5 and 15. The main symptom is ataxia, which means trouble coordinating movements. Specific symptoms include:</p><ul><li><span class="qt0">Difficulty</span> <span class="qt1">walking</span></li><li>\tMuscle weakness</li><li>\tSpeech problems</li><li>\tInvoluntary eye movements</li><li>\tScoliosis (curving of the spine to one side)</li><li>\tHeart palpitations, from the heart disease which can happen along with Friedreich ataxia</li></ul><p>People with Friedreich ataxia usually need a wheelchair 15 to 20 years after symptoms first appear. In severe cases, people become incapacitated. There is no cure. You can treat symptoms with medicines, braces, surgery, and physical therapy.</p><p>NIH: 

In [ ]:
import re
import jsonlines

titles = {}
for key in data.keys(): #new jsonl
  clean_titles = []
  clean_tag = re.compile('<.*?>')
  idx = 1

  name = f'sample_collection_jsonl_2/doc_{key}.jsonl'
  with jsonlines.open(name, 'w') as writer:
    for info in data[key]: #new dict in line
      line_dict = {}
      line_dict['id'] = 'doc' + str(idx)
      idx += 1

      title_i = info.split('\n')[0]
      url = info.split('\n')[1]
      text = info.split('\n')[2]
      clean_title = re.sub(clean_tag, '', title_i)
      clean_text = re.sub(clean_tag, '', text)
      #clean_titles.append(clean_title + '\n' + url)

      #result = url + '\n' + clean_title + '\n' + clean_text
      result = clean_title + ' ' + clean_text
      line_dict['contents'] = result
      writer.write(line_dict)

  #print(f'Doc #{key} written')
  #titles[key] = clean_titles